# QAOA Circuit Building Using QuTiP

by Pantita Palittapongarnpim

After Qiskit testing didn't pan out, we're switching to QuTiPfor simulation of QAOA circuits, and possibly for everything thereafter.

### Package Versions

QuTiP: 4.5.2

Python: 3.8.6

### Loading Packages

In [23]:
import numpy as np
import qutip as qt

### Building QAOA circuit

We'll start with a circuit of 3 qubits without the classical optimizer.

In [37]:
total_qubit=3 #number of qubit
p=5 #number of layers

#declaring QAOA parameters
gamma=np.random.rand(p)
beta=np.random.rand(p)

#defining the problem
J=np.random.rand(total_qubit,total_qubit) #creating the couping parameters,still directional in this instance)
for i in range(total_qubit):
    for j in range(i,total_qubit):
        temp=(J[i][j]+J[j][i])/2.0
        J[i][j]=temp
        J[j][i]=temp
#Now J is undirectional

Let's now define some unitary functions we are going to use.

In [4]:
def Uz(i,j,p):
    opr=qt.tensor([qt.identity(2) if (k != i and k != j) else qt.sigmaz() for k in range(total_qubit)])
    Hvar=-1j*gamma[p-1]*J[i][j]*opr
    return(Hvar.expm())

#print(Uz(0,1,1))

In [5]:
def Ux(i,p):
    opr=qt.tensor([qt.identity(2) if (k != i) else qt.sigmax() for k in range(total_qubit)])
    Hvar=-1j*beta[p-1]*J[i][j]*opr
    return(Hvar.expm())

#print(Ux(0,1))

Let's put together the circuit

In [38]:
Had=qt.tensor([qt.qip.operations.snot() for k in range(total_qubit)])

qaoa=Had

for l in range(p):
    #operate the Uz gates
    for i in range(total_qubit):
        for j in range(i,total_qubit):
            if i!= j:
                qaoa=Uz(i,j,l)*qaoa
    for i in range(total_qubit):
        qaoa=Ux(i,l)*qaoa


Now operating this unitary on the ground state.

In [39]:
gnd=qt.basis(2,0)
state=qt.tensor([gnd for i in range(total_qubit)])

state=qaoa*state

Let's do a very basic check on the state that comes out.

In [40]:
array_size=2**(total_qubit)
prob=np.zeros(array_size)

for i in range(array_size):
    prob[i]=np.abs(state[i]**2)

print(prob)
print(sum(prob))

[0.21864207 0.0799774  0.00072131 0.20065922 0.20065922 0.00072131
 0.0799774  0.21864207]
0.9999999999999988


Seems reasonable in terms of the probability sum that we got, within some rounding error.

__How do we know we are implementing it right, though?__

Observation:
 * Increase in p will increase the rounding error

### Building QAOA as a Class

__Why do we want it as a class?__

Making the circuit into a class is easier to manage in the long run as we can keep the attributes of a QAOA circuit protected and separated from another circuit quite naturally. 

In [18]:
class QAOA:
    def __init__(self, gamma, beta, p, graph):
        ## what does graph has to contain:
        # total_qubit -- integer
        # J -- double matrix        
        self.gamma=gamma
        self.beta=beta
        self.p=p
        self.J=graph.J
        self.total_qubit=graph.total_qubit
        
    def Uz(self,i,j,l):
        opr=qt.tensor([qt.identity(2) if (k != i and k != j) else qt.sigmaz() for k in range(self.total_qubit)])
        Hvar=-1j*gamma[l-1]*J[i][j]*opr
        return(Hvar.expm())
    
    def Ux(self,i,l):
        opr=qt.tensor([qt.identity(2) if (k != i) else qt.sigmax() for k in range(self.total_qubit)])
        Hvar=-1j*beta[l-1]*J[i][j]*opr
        return(Hvar.expm())
    
    def circuit(self):
        Had=qt.tensor([qt.qip.operations.snot() for k in range(self.total_qubit)])
        qaoa_circ=Had
        for l in range(self.p):
            #operate the Uz gates
            for i in range(self.total_qubit):
                for j in range(i,self.total_qubit):
                    if i!= j:
                        qaoa_circ=self.Uz(i,j,l)*qaoa_circ
            #operate the Ux gates
            for i in range(self.total_qubit):
                qaoa_circ=self.Ux(i,l)*qaoa_circ
        return(qaoa_circ)
    
    def state(self):
        gnd=qt.basis(2,0)
        state=qt.tensor([gnd for i in range(self.total_qubit)])
        state=self.circuit()*state
        return(state)
 

What we've discovered here is that the problem should be encoded as another class, we call here "graph".
So we are going to write the graph class to store the problem.

The only function I can think of right now is the Hamiltonian.

In [29]:
class Graph:
    def __init__(self, total_qubit, J):
        self.total_qubit=total_qubit #integer
        self.J=J #matrix: undirected (symmetric)
        
    def Ham(self):
        Ham_out=0;
        for i in range(self.total_qubit):
            for j in range(i,self.total_qubit):
                opr=qt.tensor([qt.identity(2) if (k != i and k != j) else qt.sigmaz() for k in range(self.total_qubit)])
                Ham_out=Ham_out+J[i][j]*opr
        return(Ham_out)

Now let's test out classes.

In [33]:
total_qubit=2 #number of qubit
p=1 #number of layers

#declaring QAOA parameters
gamma=np.random.rand(p)
beta=np.random.rand(p)

#defining the problem
J=np.random.rand(total_qubit,total_qubit) #creating the couping parameters,still directional in this instance)
for i in range(total_qubit):
    for j in range(i,total_qubit):
        temp=(J[i][j]+J[j][i])/2.0
        J[i][j]=temp
        J[j][i]=temp
#Now J is undirectional

In [34]:
graph = Graph(total_qubit,J)
qaoa = QAOA(gamma,beta,p,graph)

print(graph.Ham())

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 2.21681915  0.          0.          0.        ]
 [ 0.         -0.59215824  0.          0.        ]
 [ 0.          0.         -0.98331469  0.        ]
 [ 0.          0.          0.         -0.64134623]]


In [35]:
output=qaoa.state()

In [36]:
array_size=2**(total_qubit)
prob=np.zeros(array_size)

for i in range(array_size):
    prob[i]=np.abs(output[i]**2)

print(prob)
print(sum(prob))

[0.44859616 0.05140384 0.05140384 0.44859616]
0.9999999999999993
